In [5]:
# import external packages
import numpy as np
import pandas as pd
import numba
from numba import vectorize
import glob # for file search
import copy
import os # operating system stuff
import re # regex
import fastparquet # fast read/write for large data structures
import sklearn.preprocessing as pre # for data normalisation
from sklearn.metrics import pairwise_distances

import geopandas as gpd
import rasterio as rio
import rasterio.mask
from rasterio.plot import plotting_extent
from shapely.geometry import Polygon
from shapely.geometry.point import Point
import pyproj
from pyproj import CRS
from inpoly import inpoly2 # for fast inpolygon checks
import utm

import matplotlib.pyplot as plt 
import matplotlib.dates as mdates
from matplotlib import cm as mpl_cm
from matplotlib import colors as mcolors 

from mpl_toolkits.axes_grid1 import make_axes_locatable # for colorbar scaling
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib_scalebar.scalebar import ScaleBar
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import FormatStrFormatter

import seaborn as sns
from matplotlib import rc_file_defaults
rc_file_defaults()
# sns.set(style=None, color_codes=True)

from shapely.geometry import Polygon
from shapely.geometry.point import Point
import datetime

import configparser

from cmcrameri import cm # for scientific colourmaps

###########################
# import main local package
import SPOTSAR_main as sm


In [6]:
################ Define user INPUTS #######################
######## please edit the values of this block only ########
###########################################################

# define hillshade file
HS_FILE = './test_data/DEM/TDX_Merapi_WGS84_HS.tif'

# define lon and lat files
LON_FILE = './test_data/CSK_dsc/geo2/20200910.lon'
LAT_FILE = './test_data/CSK_dsc/geo2/20200910.lat'

# define parameter text file
PARAM_FILE = './test_data/CSK_dsc/params.txt'

# define map region of interest
lon_lims = [110.425, 110.45]
lat_lims = [-7.555, -7.535]

# define colour range {min max} (min = -max)
vmax = 3 # range of colourscale in meters

# define file names for data, lon and lat
DIRECTORY_PATH = "./test_data/CSK_dsc/DISP_txt2/"
# define path to ccp and ccs files
DIRECTORY_PATH_CCS = "./test_data/CSK_dsc/CCS2/"

# Set the regular expression pattern to match the file names
PATTERN1 = r"^c20200927_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN2 = r"^c20200926_c20201113_disp_[0-9]+_[0-9]+\.txt$"
PATTERN3 = r"^c20200927_c20210812_disp_[0-9]+_[0-9]+\.txt$"
PATTERN4 = r"^c20210217_c20210218_disp_[0-9]+_[0-9]+\.txt$"

# Set the regular expression pattern to match the ccs file names
PATTERN_CCS1 = r"^c20200927_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS2 = r"^c20200926_c20201113_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS3 = r"^c20200927_c20210812_ccs_[0-9]+_[0-9]+$"
PATTERN_CCS4 = r"^c20210217_c20210218_ccs_[0-9]+_[0-9]+$"


# open hillshade file and re-order offset and CCS files

# open hill shade file with rasterio
DEM_HS = rio.open(HS_FILE)
SHADING = DEM_HS.read(1,masked=True) # rasterio bands are indexed from 1

# extract DEM extent
DEM_EXTENT=[DEM_HS.bounds.left,DEM_HS.bounds.right,DEM_HS.bounds.bottom,DEM_HS.bounds.top]

# read parameters from text file
config = configparser.ConfigParser()
config.read(PARAM_FILE)
WIDTH = int(config.get('params', 'width'))
LINES = int(config.get('params', 'lines'))
WIDTH_CCS = int(config.get('params', 'width_ccs'))
LINES_CCS = int(config.get('params', 'lines_ccs'))
R_START = int(config.get('params', 'r_start'))
A_START = int(config.get('params', 'a_start'))
R_STEP = int(config.get('params', 'r_step'))
A_STEP = int(config.get('params', 'a_step'))
HEADING = float(config.get('params', 'heading'))
MEAN_INC = float(config.get('params', 'mean_inc'))

In [7]:

# reorder file using Post_processing.reorder_files
matching_files1 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN1,0)
matching_files_ccs1 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS1,0)
matching_files2 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN2,0)
matching_files_ccs2 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS2,0)
matching_files3 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN3,0)
matching_files_ccs3 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS3,0)
matching_files4 = sm.Post_processing.reorder_files(DIRECTORY_PATH,PATTERN4,0)
matching_files_ccs4 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS,PATTERN_CCS4,0)
matching_files5 = sm.Post_processing.reorder_files(DIRECTORY_PATH+'stor/',PATTERN4,0)
matching_files_ccs5 = sm.Post_processing.reorder_files(DIRECTORY_PATH_CCS+'stor/',PATTERN_CCS4,0)


# test if file ordering has worked
print(matching_files1)
print(matching_files_ccs1)
print(matching_files2)
print(matching_files_ccs2)
print(matching_files3)
print(matching_files_ccs3)
print(matching_files4)
print(matching_files_ccs4)
print(matching_files5)
print(matching_files_ccs5)

['c20200927_c20201113_disp_58_28.txt', 'c20200927_c20201113_disp_140_68.txt', 'c20200927_c20201113_disp_224_108.txt', 'c20200927_c20201113_disp_306_148.txt', 'c20200927_c20201113_disp_388_188.txt']
['c20200927_c20201113_ccs_58_28', 'c20200927_c20201113_ccs_140_68', 'c20200927_c20201113_ccs_224_108', 'c20200927_c20201113_ccs_306_148', 'c20200927_c20201113_ccs_388_188']
['c20200926_c20201113_disp_58_28.txt', 'c20200926_c20201113_disp_140_68.txt', 'c20200926_c20201113_disp_224_108.txt', 'c20200926_c20201113_disp_306_148.txt', 'c20200926_c20201113_disp_388_188.txt']
['c20200926_c20201113_ccs_58_28', 'c20200926_c20201113_ccs_140_68', 'c20200926_c20201113_ccs_224_108', 'c20200926_c20201113_ccs_306_148', 'c20200926_c20201113_ccs_388_188']
['c20200927_c20210812_disp_58_28.txt', 'c20200927_c20210812_disp_140_68.txt', 'c20200927_c20210812_disp_224_108.txt', 'c20200927_c20210812_disp_306_148.txt', 'c20200927_c20210812_disp_388_188.txt']
['c20200927_c20210812_ccs_58_28', 'c20200927_c20210812_ccs_1

In [8]:

# load data from files into class multi-kernel
example_pairs = []
i = 0
for (matching_files,matching_files_ccs) in zip([matching_files1,matching_files2,matching_files3,matching_files4,matching_files5],[matching_files_ccs1,matching_files_ccs2,matching_files_ccs3,matching_files_ccs4,matching_files_ccs5]):
    if i==4:
        datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH+'stor/',
                                                matching_files,
                                                DIRECTORY_PATH_CCS+'stor/',
                                                matching_files_ccs,
                                                LAT_FILE,
                                                LON_FILE,
                                                HEADING,
                                                MEAN_INC,
                                                LINES_CCS,
                                                WIDTH_CCS)
    else:
        datastack = sm.Post_processing.MultiKernel(DIRECTORY_PATH,
                                                matching_files,
                                                DIRECTORY_PATH_CCS,
                                                matching_files_ccs,
                                                LAT_FILE,
                                                LON_FILE,
                                                HEADING,
                                                MEAN_INC,
                                                LINES_CCS,
                                                WIDTH_CCS)

    # We need to assign some data not stored in the disp.txt files.
    datastack.get_params_from_file_name()
    datastack.get_latlon_from_file(WIDTH)
    datastack.add_lat_lon_to_data(R_START,A_START)
    datastack.crop_stack_ccs(R_STEP,A_STEP)
    # the object datastack now has several attributes associated with the whole dataset (e.g., date1, date2, heading)
    # Next we add all the offset data (disp.txt) to the stack
    stacked_data = datastack.assign_data_to_stack(R_STEP,A_STEP)
    # The attribute 'Stack' we find a list of single-kernel objects which contain the actual offset data, ccp and ccs data and the coordinates.

    # add stack to list
    example_pairs.append(datastack)
    i +=1



In [9]:
# %matplotlib osx
# from scipy.ndimage import generic_filter
# from skimage.morphology import disk
# from skimage import filters
# from scipy.spatial import cKDTree

# filt_size = 3
# filt_radius = 3
# footprint = disk(radius=filt_radius)
# pre_R_non_veg_list = []
# pre_R_veg_list = []
# pre_A_non_veg_list = []
# pre_A_veg_list = []

# for obj in example_pairs[3].Stack:
#     R_off = obj.R_off
#     A_off = obj.A_off

#     print('starting filters')

#     R_off_std = generic_filter(R_off, np.nanstd, footprint=footprint)
#     A_off_std = generic_filter(A_off, np.nanstd, footprint=footprint)

#     print('filters done')


#     DEM_lon_lat = np.column_stack((DEM_lons.flatten(),DEM_lats.flatten()))
#     NDVI_lon_lat = np.column_stack((NDVI_lons.flatten(),NDVI_lats.flatten()))
#     data_lon_lat = np.column_stack((obj.Lon_off.flatten(),obj.Lat_off.flatten()))

#     print('build trees')
#     kdtree_dem = cKDTree(DEM_lon_lat)
#     kdtree_ndvi = cKDTree(NDVI_lon_lat)
#     print('done building trees')

#     data_dem = np.empty(np.shape(R_off.flatten()))
#     data_slope = np.empty(np.shape(R_off.flatten()))
#     data_ndvi = np.empty(np.shape(R_off.flatten()))
#     idx_list = []
#     for i, point in enumerate(data_lon_lat):
#         if not all(np.isnan(point)):
#             print(i)
#             distance,index_DEM = kdtree_dem.query(point)
#             distance,index_NDVI = kdtree_ndvi.query(point)
#             idx_list.append([i,index_DEM,index_NDVI])

#     idx_list = np.asarray(idx_list)
#     data_dem[idx_list[:,0]] = DEM_heights.flatten()[idx_list[:,1]]
#     data_slope[idx_list[:,0]] = DEM_slope.flatten()[idx_list[:,1]]
#     data_ndvi[idx_list[:,0]] = NDVI.flatten()[idx_list[:,2]]
#     data_ndvi[data_ndvi<-1] = np.nan

#     R_sel_non_veg = R_off_std.flatten()[np.argwhere(np.abs(data_ndvi) <= 0.1)]
#     A_sel_non_veg = A_off_std.flatten()[np.argwhere(np.abs(data_ndvi) <= 0.1)]
#     R_sel_veg = R_off_std.flatten()[np.argwhere(data_ndvi >= 0.5)]
#     A_sel_veg = A_off_std.flatten()[np.argwhere(data_ndvi >= 0.5)]

#     R_data_non_veg = R_sel_non_veg.flatten()
#     R_data_veg = R_sel_veg.flatten()
#     R_data_non_veg[R_data_non_veg <= 0] = np.nan
#     R_data_veg[R_data_veg <= 0] = np.nan
#     A_data_non_veg = A_sel_non_veg.flatten()
#     A_data_veg = A_sel_veg.flatten()
#     A_data_non_veg[A_data_non_veg <= 0] = np.nan
#     A_data_veg[A_data_veg <= 0] = np.nan

#     pre_R_non_veg_list.append(R_data_non_veg)
#     pre_R_veg_list.append(R_data_veg)
#     pre_A_non_veg_list.append(A_data_non_veg)
#     pre_A_veg_list.append(A_data_veg)


In [10]:
# load med filt and MKA results from h5 files

import h5py 

# file1 contains window sizes 58, 140, 224, 306, 388, MKA 1-5, MKA 1-3
file1 = './test_data/CSK_dsc/filtered_MKA_offsets/c20210217_c20210218_v3.h5'

# file2 contains window sizes 58, 99, 140, 182, 224, MKA 1-5
file2 = './test_data/CSK_dsc/filtered_MKA_offsets/c20210217_c20210218_v4.h5'

# provide attribute names
attr_names1 = ['win_1_Lon', 'win_1_lat', 'win_1_R_off', 'win_1_A_off',
               'win_2_Lon', 'win_2_lat', 'win_2_R_off', 'win_2_A_off',
               'win_3_Lon', 'win_3_lat', 'win_3_R_off', 'win_3_A_off',
               'win_4_Lon', 'win_4_lat', 'win_4_R_off', 'win_4_A_off',
               'win_5_Lon', 'win_5_lat', 'win_5_R_off', 'win_5_A_off',
               'win_13_Lon', 'win_13_lat', 'win_13_R_off', 'win_13_A_off',
               'win_15_Lon', 'win_15_lat', 'win_15_R_off', 'win_15_A_off',]

attr_names2 = ['win_1_Lon', 'win_1_lat', 'win_1_R_off', 'win_1_A_off',
               'win_2_Lon', 'win_2_lat', 'win_2_R_off', 'win_2_A_off',
               'win_3_Lon', 'win_3_lat', 'win_3_R_off', 'win_3_A_off',
               'win_4_Lon', 'win_4_lat', 'win_4_R_off', 'win_4_A_off',
               'win_5_Lon', 'win_5_lat', 'win_5_R_off', 'win_5_A_off',
               'win_15_Lon', 'win_15_lat', 'win_15_R_off', 'win_15_A_off',]

# define example objects (both pair 4)
obj1 = example_pairs[3]
obj2 = example_pairs[4]

for i, (file, obj,attr_names) in enumerate(zip([file1,file2],[obj1,obj2],[attr_names1,attr_names2])):
    f = h5py.File(file,'r')
    for qkey in attr_names:
        attr = np.array(f[f'{qkey}'][:])
        # attr = f.get(f'{qkey}')
        setattr(obj,f'{qkey}',attr)
    f.close() 

In [11]:
# check if attributes are loaded correctly

print(dir(obj2))

['A_win', 'Ccs_maps', 'Data', 'Data_ccs', 'Date1', 'Date2', 'Filenames', 'Filenames_ccs', 'Heading', 'Lat', 'Lat_file', 'Lat_vec', 'Limits', 'Lon', 'Lon_file', 'Lon_vec', 'Mask_data', 'Mask_data_ccs', 'Mean_inc', 'R_win', 'Run_MKA', 'Run_RSS', 'Stack', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'add_lat_lon_to_data', 'assign_data_to_stack', 'crop_stack_ccs', 'get_latlon_from_file', 'get_params_from_file_name', 'query_point_MKA', 'query_point_stack', 'win_15_A_off', 'win_15_Lon', 'win_15_R_off', 'win_15_lat', 'win_1_A_off', 'win_1_Lon', 'win_1_R_off', 'win_1_lat', 'win_2_A_off', 'win_2_Lon', 'win_2_R_off', 'win_2_lat', 'win_3_A_off', 'win_3_Lon', 'win_3_R_off', 'win_3_lat', 'win_4_A_off', 'w

In [12]:
print(obj2.win_1_Lon)

[[110.48296356 110.48293304 110.48291779 ... 110.47357941 110.47362518
  110.47364807]
 [110.4827652  110.48277283 110.4827652  ... 110.47351837 110.47346497
  110.47341156]
 [110.48258209 110.48256683 110.48257446 ... 110.4732132  110.47316742
  110.47318268]
 ...
 [110.42795563 110.42786407 110.42780304 ...          nan          nan
           nan]
 [110.42781067 110.42771912 110.42771912 ...          nan 110.41773224
  110.41771698]
 [110.4276886           nan 110.42755127 ...          nan 110.41762543
  110.41763306]]


In [13]:
# load external data

import fiona
from matplotlib.colors import ListedColormap
from matplotlib import cm as mpl_cm
## find nearest elevation, slope, and NDVI values
DEM_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m.tif'
SLOPE_FILE = './test_data/DEM/TDX_Merapi_WGS84_5m_slope.tif'
NDVI_FILE = '/Users/markbemelmans/Documents/PhD/projects/Merapi2021/sentinel2/Sentinel_NDVI_WGS84_v2.tif'

with rio.open(DEM_FILE) as src:
    DEM_heights = src.read(1)
    print('Band1 has shape', DEM_heights.shape)
    height = DEM_heights.shape[0]
    width = DEM_heights.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    DEM_lons = np.array(xs)
    DEM_lats = np.array(ys)
    DEM_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

with rio.open(SLOPE_FILE) as src:
    DEM_slope = src.read(1)


# define map region of interest
lon_lims = [np.nanmin(DEM_lons), np.nanmax(DEM_lons)]
lat_lims = [np.nanmin(DEM_lats), np.nanmax(DEM_lats)]
crop_flag=1

# create cropping polygon from ROI
if crop_flag:
    coords = ((lon_lims[0], lat_lims[0]), (lon_lims[0], lat_lims[1]), (lon_lims[1], lat_lims[1]), (lon_lims[1], lat_lims[0]), (lon_lims[0], lat_lims[0]))
    crop_poly = Polygon(coords)
    crop_poly_geojson = gpd.GeoSeries([crop_poly])
    crop_poly_geojson.to_file('./test_data/crop_ndvi_poly.shp',crs="EPSG:4326")


with fiona.open('./test_data/crop_ndvi_poly.shp', "r") as shapefile:
    shapes = [feature["geometry"] for feature in shapefile]

# get data and read coords from first file
with rio.open(NDVI_FILE) as src:
    out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
    out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform})
with rasterio.open("./test_Data/ndvi_cropped.tif", "w", **out_meta) as dest:
    dest.write(out_image)

with rio.open("./test_Data/ndvi_cropped.tif") as src:
    NDVI = src.read(1)
    print('Band1 has shape', NDVI.shape)
    height = NDVI.shape[0]
    width = NDVI.shape[1]
    cols, rows = np.meshgrid(np.arange(width), np.arange(height))
    xs, ys = rio.transform.xy(src.transform, rows, cols)
    NDVI_lons = np.array(xs)
    NDVI_lats = np.array(ys)
    NDVI_extent = [src.bounds.left,src.bounds.right,src.bounds.bottom,src.bounds.top]

# define colormaps
x= mpl_cm.get_cmap('Blues_r', 135)
y= mpl_cm.get_cmap('YlGn', 135)
z = np.vstack((x(range(135)),
                       y(range(135))))
ndvi_cmap = ListedColormap(z, name='BlYlGn')

Band1 has shape (2028, 1672)
Band1 has shape (171, 141)


/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_90318/1832665036.py:65: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  x= mpl_cm.get_cmap('Blues_r', 135)
/var/folders/m9/32v45xkn4zx6js6htb1prb2r0000gn/T/ipykernel_90318/1832665036.py:66: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  y= mpl_cm.get_cmap('YlGn', 135)


In [14]:
# Apply std filter

%matplotlib osx
from scipy.ndimage import generic_filter
from skimage.morphology import disk
from skimage import filters
from scipy.spatial import cKDTree


DEM_lon_lat = np.column_stack((DEM_lons.flatten(),DEM_lats.flatten()))
NDVI_lon_lat = np.column_stack((NDVI_lons.flatten(),NDVI_lats.flatten()))
print('build trees')
kdtree_dem = cKDTree(DEM_lon_lat)
kdtree_ndvi = cKDTree(NDVI_lon_lat)
print('done building trees')


filt_size = 3
filt_radius = 3
footprint = disk(radius=filt_radius)
R_non_veg_list = []
R_veg_list = []
A_non_veg_list = []
A_veg_list = []

obj_list = [obj1.Stack[0], obj1, 
            obj1.Stack[1], obj1, 
            obj1.Stack[2], obj1, 
            obj1.Stack[3], obj1, 
            obj1.Stack[4], obj1, 
            obj1, obj1,
            obj2.Stack[0], obj2, 
            obj2.Stack[1], obj2, 
            obj2.Stack[2], obj2, 
            obj2.Stack[3], obj2, 
            obj2.Stack[4], obj2, 
            obj2]
R_off_list = ['R_off', 'win_1_R_off', 
              'R_off', 'win_2_R_off', 
              'R_off', 'win_3_R_off', 
              'R_off', 'win_4_R_off',
              'R_off', 'win_5_R_off',
              'win_13_R_off','win_15_R_off',
              'R_off', 'win_1_R_off', 
              'R_off', 'win_2_R_off',
              'R_off', 'win_3_R_off',
              'R_off', 'win_4_R_off',
              'R_off', 'win_5_R_off',
              'win_15_R_off']
A_off_list = ['A_off', 'win_1_A_off', 
              'A_off', 'win_2_A_off', 
              'A_off', 'win_3_A_off', 
              'A_off', 'win_4_A_off',
              'A_off', 'win_5_A_off',
              'win_13_A_off','win_15_A_off',
              'A_off', 'win_1_A_off', 
              'A_off', 'win_2_A_off',
              'A_off', 'win_3_A_off',
              'A_off', 'win_4_A_off',
              'A_off', 'win_5_A_off',
              'win_15_A_off']

Lon_off_list = ['Lon_off', 'win_1_Lon', 
                'Lon_off', 'win_2_Lon', 
                'Lon_off', 'win_3_Lon', 
                'Lon_off', 'win_4_Lon',
                'Lon_off', 'win_5_Lon',
                'win_13_Lon','win_15_Lon',
                'Lon_off', 'win_1_Lon', 
                'Lon_off', 'win_2_Lon',
                'Lon_off', 'win_3_Lon',
                'Lon_off', 'win_4_Lon',
                'Lon_off', 'win_5_Lon',
                'win_15_Lon']

Lat_off_list = ['Lat_off', 'win_1_lat', 
                'Lat_off', 'win_2_lat', 
                'Lat_off', 'win_3_lat', 
                'Lat_off', 'win_4_lat',
                'Lat_off', 'win_5_lat',
                'win_13_lat','win_15_lat',
                'Lat_off', 'win_1_lat', 
                'Lat_off', 'win_2_lat',
                'Lat_off', 'win_3_lat',
                'Lat_off', 'win_4_lat',
                'Lat_off', 'win_5_lat',
                'win_15_lat']


for (obj, R_off_name, A_off_name, Lon_off_name, Lat_off_name) in zip(obj_list, R_off_list, A_off_list, Lon_off_list, Lat_off_list):
    # get data
    R_off = getattr(obj,R_off_name)
    A_off = getattr(obj,A_off_name)
    Lon_off = getattr(obj,Lon_off_name)
    Lat_off = getattr(obj,Lat_off_name)


    print('starting filters')

    R_off_std = generic_filter(R_off, np.nanstd, footprint=footprint)
    A_off_std = generic_filter(A_off, np.nanstd, footprint=footprint)

    print('filters done')

    # get latlon data in column format
    data_lon_lat = np.column_stack((Lon_off.flatten(),Lat_off.flatten()))

    # create empty data to match length of R_off
    data_dem = np.empty(np.shape(R_off.flatten()))
    data_slope = np.empty(np.shape(R_off.flatten()))
    data_ndvi = np.empty(np.shape(R_off.flatten()))
    idx_list = []
    # get closest coordinate
    for i, point in enumerate(data_lon_lat):
        if not all(np.isnan(point)):
            # if np.mod(i,50000)==0:
            #     print(i)
            distance,index_DEM = kdtree_dem.query(point)
            distance,index_NDVI = kdtree_ndvi.query(point)
            idx_list.append([i,index_DEM,index_NDVI])

    # get nearest neighbor external data
    idx_list = np.asarray(idx_list)
    data_dem[idx_list[:,0]] = DEM_heights.flatten()[idx_list[:,1]]
    data_slope[idx_list[:,0]] = DEM_slope.flatten()[idx_list[:,1]]
    data_ndvi[idx_list[:,0]] = NDVI.flatten()[idx_list[:,2]]
    data_ndvi[data_ndvi<-1] = np.nan

    # get matching std data
    R_sel_non_veg = R_off_std.flatten()[np.argwhere(np.abs(data_ndvi) <= 0.1)]
    A_sel_non_veg = A_off_std.flatten()[np.argwhere(np.abs(data_ndvi) <= 0.1)]
    R_sel_veg = R_off_std.flatten()[np.argwhere(data_ndvi >= 0.5)]
    A_sel_veg = A_off_std.flatten()[np.argwhere(data_ndvi >= 0.5)]

    # condition data
    R_data_non_veg = R_sel_non_veg.flatten()
    R_data_veg = R_sel_veg.flatten()
    R_data_non_veg[R_data_non_veg <= 0] = np.nan
    R_data_veg[R_data_veg <= 0] = np.nan
    A_data_non_veg = A_sel_non_veg.flatten()
    A_data_veg = A_sel_veg.flatten()
    A_data_non_veg[A_data_non_veg <= 0] = np.nan
    A_data_veg[A_data_veg <= 0] = np.nan

    # add data to list
    R_non_veg_list.append(R_data_non_veg)
    R_veg_list.append(R_data_veg)
    A_non_veg_list.append(A_data_non_veg)
    A_veg_list.append(A_data_veg)



build trees
done building trees
starting filters


/Applications/anaconda3/envs/PhD/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done
starting filters
filters done


In [15]:
print(A_non_veg_list)

[array([0.14502045, 0.03661825, 0.0403272 , ..., 0.90415036, 0.44362847,
       0.44362847]), array([0.14502045, 0.03661825, 0.0403272 , ..., 0.44362847, 0.44362847,
       0.44362847]), array([0.00804949, 0.00909953, 0.01143716, ..., 0.33976407, 0.51341951,
       0.55191695]), array([0.00804949, 0.00909953, 0.01143716, ..., 0.33976407, 0.51341951,
       0.48974998]), array([0.00615592, 0.00826752, 0.0087631 , ..., 0.03969372, 0.03977481,
       0.18461497]), array([0.00615592, 0.00826752, 0.0087631 , ..., 0.03969372, 0.03977481,
       0.18461497]), array([0.0024587 , 0.00589726, 0.00368659, ..., 0.00924422, 0.00990504,
       0.00760683]), array([0.0024587 , 0.00589726, 0.00368659, ..., 0.00924422, 0.00990504,
       0.00760683]), array([0.0018805 , 0.00338458, 0.00476761, ..., 0.00315385, 0.00264082,
       0.00202973]), array([0.0018805 , 0.00338458, 0.00476761, ..., 0.00315385, 0.00264082,
       0.00202973]), array([0.00451622, 0.00715172, 0.00585866, ..., 0.01580946, 0.0143332

In [16]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 12})

# Sample data (replace with your actual data)
# R_group_A = [
#     R_non_veg_list[0][~np.isnan(R_non_veg_list[0])],
#     R_non_veg_list[1][~np.isnan(R_non_veg_list[1])],
#     R_non_veg_list[2][~np.isnan(R_non_veg_list[2])],
#     R_non_veg_list[3][~np.isnan(R_non_veg_list[3])],
#     R_non_veg_list[4][~np.isnan(R_non_veg_list[4])],
#     R_non_veg_list[5][~np.isnan(R_non_veg_list[5])],
# ]
dec_factor = 100
indeces = [0,1,2,3,4,5,6,7,8,9,10,11,]
positions = [1,2, 
             4-0.5,5-0.5, 
             7-1,8-1, 
             10-1.5,11-1.5, 
             13-2,14-2, 
             16-2.5,17-2.5]
labels_r = ['58 ', '58 ',
            '140', '140', 
            '224', '224',
            '306', '306',
            '388', '388',
            'MKA\n1-5', 'MKA\n1-3']

labels_a = ['28 ', '28 ',
            '68 ', '68 ', 
            '108', '108',
            '148', '148',
            '188', '188',
            'MKA\n1-5', 'MKA\n1-3']
# 12,13,14,15,16,17,18,19,20,21,22]
R_group_A = [R_non_veg_list[i][~np.isnan(R_non_veg_list[i])][::dec_factor] for i in indeces]
R_group_B = [R_veg_list[i][~np.isnan(R_veg_list[i])][::dec_factor] for i in indeces]
A_group_A = [A_non_veg_list[i][~np.isnan(A_non_veg_list[i])][::dec_factor] for i in indeces]
A_group_B = [A_veg_list[i][~np.isnan(A_veg_list[i])][::dec_factor] for i in indeces]

# R_group_B = [
#     R_veg_list[0][~np.isnan(R_veg_list[0])],
#     R_veg_list[1][~np.isnan(R_veg_list[1])],
#     R_veg_list[2][~np.isnan(R_veg_list[2])],
#     R_veg_list[3][~np.isnan(R_veg_list[3])],
#     R_veg_list[4][~np.isnan(R_veg_list[4])],
#     R_veg_list[5][~np.isnan(R_veg_list[5])],
# ]

# A_group_A = [
#     A_non_veg_list[0][~np.isnan(A_non_veg_list[0])],
#     A_non_veg_list[1][~np.isnan(A_non_veg_list[1])],
#     A_non_veg_list[2][~np.isnan(A_non_veg_list[2])],
#     A_non_veg_list[3][~np.isnan(A_non_veg_list[3])],
#     A_non_veg_list[4][~np.isnan(A_non_veg_list[4])],
#     A_non_veg_list[5][~np.isnan(A_non_veg_list[5])],
# ]

# A_group_B = [
#     A_veg_list[0][~np.isnan(A_veg_list[0])],
#     A_veg_list[1][~np.isnan(A_veg_list[1])],
#     A_veg_list[2][~np.isnan(A_veg_list[2])],
#     A_veg_list[3][~np.isnan(A_veg_list[3])],
#     A_veg_list[4][~np.isnan(A_veg_list[4])],
#     A_veg_list[5][~np.isnan(A_veg_list[5])],
# ]

# define colours
colors_pre = sns.color_palette("Paired")[0:10] # for the window sizes
colors = [colors_pre[0],colors_pre[1],
          colors_pre[6],colors_pre[7],
          colors_pre[2],colors_pre[3],
          colors_pre[4],colors_pre[5],
          colors_pre[8],colors_pre[9]]
colors.append((0,0,0))
colors.append((0.5,0.5,0.5)) # for MKa results


# define median line props
medianprops = dict(linestyle='-', linewidth=2.5, color='white')
# define mean marker
meanpointprops = dict(marker='^', markeredgecolor='black',
                      markerfacecolor='white')

# Create subplots for box plots
fig, axs = plt.subplots(2, 2, figsize=(26, 10))

# Create box plots for group A
box = axs[0,0].boxplot(R_group_A,showmeans=True,notch=False, patch_artist=True,positions=positions,labels= labels_r,medianprops=medianprops,meanprops=meanpointprops,whis = [2.5,97.5],sym='k.')
axs[0,0].set_title("Range offset non-vegetated")
axs[0,0].set_yscale('log')

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)

# Create box plots for group B
box = axs[0,1].boxplot(R_group_B,showmeans=True,notch=False, patch_artist=True,positions=positions,labels= labels_r,medianprops=medianprops,meanprops=meanpointprops,whis = [2.5,97.5],sym='k.')
axs[0,1].set_title("Range offset vegetated")
axs[0,1].set_yscale('log')

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)


# Create box plots for group A
box = axs[1,0].boxplot(A_group_A,showmeans=True,notch=False, patch_artist=True,positions=positions,labels= labels_a,medianprops=medianprops,meanprops=meanpointprops,whis = [2.5,97.5],sym='k.')
axs[1,0].set_title("Azimuth offset non-vegetated")
axs[1,0].set_yscale('log')

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)


# Create box plots for group B
box = axs[1,1].boxplot(A_group_B,showmeans=True,notch=False, patch_artist=True,positions=positions,labels= labels_a,medianprops=medianprops,meanprops=meanpointprops,whis = [2.5,97.5],sym='k.')
axs[1,1].set_title("Azimuth offset vegetated")
axs[1,1].set_yscale('log')

for patch, color in zip(box['boxes'], colors):
    patch.set_facecolor(color)



# Add labels and title to the entire figure
for (a,s,t) in zip(axs.ravel(),[0.34,0.34,0.7,0.7],['A','B','C','D']):
    a.set_xlabel("Window sizes [pixels]")
    a.set_ylabel("Standard deviation [m]")
    a.set_ylim([0.0001,10])
    a.annotate(t, xy=[0.02,0.02], xytext=[0.02,0.02], xycoords='axes fraction')

    right_axis = a.twinx()
    right_axis.set_ylim(a.get_ylim()[0]/s, a.get_ylim()[1]/s)
    right_axis.spines['right'].set_position(('outward', 0))
    right_axis.spines['right'].set_linewidth(0.5)
    right_axis.set_ylabel("Pixel fraction [-]")  # Label for the right axis
    right_axis.set_yscale('log')
    a.set_yscale('log')
fig.tight_layout()
# Display the plot
# plt.show()

plt.savefig('/Users/markbemelmans/Documents/PhD/projects/Merapi2021/figures/all_boxplots.pdf')

# fig, axs = plt.subplots(2, 2, figsize=(10, 10))

# # Create box plots for group A
# axs[0,0].scatter(range(6),[np.nansum(i)/np.sum(~np.isnan(i)) for i in R_group_A])
# axs[0,0].set_title("Range offset non-vegetated")
# # axs[0,0].set_yscale('log')

# # Create box plots for group B
# axs[0,1].scatter(range(6),[np.nansum(i)/np.sum(~np.isnan(i)) for i in R_group_B])
# axs[0,1].set_title("Range offset vegetated")
# # axs[0,1].set_yscale('log')

# # Create box plots for group A
# axs[1,0].scatter(range(6),[np.nansum(i)/np.sum(~np.isnan(i)) for i in A_group_A])
# axs[1,0].set_title("Azimuth offset non-vegetated")
# # axs[1,0].set_yscale('log')

# # Create box plots for group B
# axs[1,1].scatter(range(6),[np.nansum(i)/np.sum(~np.isnan(i)) for i in A_group_B])
# axs[1,1].set_title("Azimuth offset vegetated")
# axs[1,1].set_yscale('log')


In [17]:
print(sns.color_palette("Paired")[0:10])

[(0.6509803921568628, 0.807843137254902, 0.8901960784313725), (0.12156862745098039, 0.47058823529411764, 0.7058823529411765), (0.6980392156862745, 0.8745098039215686, 0.5411764705882353), (0.2, 0.6274509803921569, 0.17254901960784313), (0.984313725490196, 0.6039215686274509, 0.6), (0.8901960784313725, 0.10196078431372549, 0.10980392156862745), (0.9921568627450981, 0.7490196078431373, 0.43529411764705883), (1.0, 0.4980392156862745, 0.0), (0.792156862745098, 0.6980392156862745, 0.8392156862745098), (0.41568627450980394, 0.23921568627450981, 0.6039215686274509)]


1   HIToolbox                           0x00007ff821cb9726 _ZN15MenuBarInstance22EnsureAutoShowObserverEv + 102
2   HIToolbox                           0x00007ff821cb92b8 _ZN15MenuBarInstance14EnableAutoShowEv + 52
3   HIToolbox                           0x00007ff821c28cd7 _ZN15MenuBarInstance21UpdateAggregateUIModeE21MenuBarAnimationStylehhh + 1113
4   HIToolbox                           0x00007ff821cb9173 _ZN15MenuBarInstance19SetFullScreenUIModeEjj + 175
5   AppKit                              0x00007ff81b786287 -[NSApplication _setPresentationOptions:instance:flags:] + 1145
6   AppKit                              0x00007ff81b5db055 -[NSApplication _updateFullScreenPresentationOptionsForInstance:] + 582
7   AppKit                              0x00007ff81c0297f0 -[_NSFullScreenSpace(PresentationInstance) activateFullScreenPresentationOptions] + 207
8   AppKit                              0x00007ff81be7c5d8 -[_NSEnterFullScreenTransitionController _doSucceededToEnterFullScreen] + 721


: 